In [ ]:
import requests
import pandas as pd

tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=0xb1332307bd818db68e372269fd515f6523cd4bce&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
token_url = "https://api.etherscan.io/api?module=account&action=tokennfttx&address=0xb1332307bd818db68e372269fd515f6523cd4bce&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
    
#tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=0x9bc27a47b4413c51f884ae4e4b9170f3a0d7f742&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
#token_url = "https://api.etherscan.io/api?module=account&action=tokennfttx&address=0x9bc27a47b4413c51f884ae4e4b9170f3a0d7f742&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"


tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=0x0239769a1adf4def9f07da824b80b9c4fcb59593&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
token_url = "https://api.etherscan.io/api?module=account&action=tokennfttx&address=0x0239769a1adf4def9f07da824b80b9c4fcb59593&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
   
    

    

In [ ]:
response = requests.get(tx_url)
data = response.json()
txlist_df = pd.DataFrame(data["result"])
txlist_df

In [ ]:
response = requests.get(token_url)
data = response.json()
tokennfttx_df = pd.DataFrame(data["result"])
tokennfttx_df

In [ ]:
txlist_df

In [ ]:
tokennfttx_df

In [ ]:
tokennfttx_df.columns

In [ ]:
df = txlist_df.merge(tokennfttx_df, how='inner', on=["timeStamp"])
df

In [ ]:
df1 = pd.DataFrame(df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]].groupby(["timeStamp", "tokenSymbol"])["tokenID"].apply(list)).reset_index()


df2 = df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]].drop_duplicates(subset=['timeStamp'], keep='last').reset_index()

#[''].apply(list)

In [ ]:
df = pd.merge(df1,df2,how="inner", on="timeStamp")
df = df.rename(columns={"tokenSymbol_x": "tokenSymbol", "tokenID_x": "tokenID", "tokenSymbol_y": "tokenSymbol", "gas_x": "gas", "gasPrice_x": "gasPrice", "gasUsed_x": "gasUsed" })
df

In [ ]:
#df = df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]]


#.drop_duplicates(subset=['timeStamp'], keep='last')




In [ ]:
df.reset_index()

In [ ]:
df["value"] = df["value"].astype(int)/1000000000000000000
 
# /1000000000000000000
df

In [ ]:
total = sum(df["value"])
total

In [ ]:
#0.003577626003325 Ether 
4230000000000000000/1000000000000000000

In [ ]:
sum(df["value"].astype(int)/1000000000000000000)


In [ ]:
gas = sum((df["gasPrice"].astype(int) * df["gasUsed"].astype(int))/1000000000000000000)
print(gas)
print(gas + total)

In [ ]:
sum(df[df["isError"] == "1"]["gasPrice"].astype(int) *  df[df["isError"] == "1"]["gasUsed"].astype(int))/1000000000000000000

In [ ]:
max(df["value"].astype(int)/1000000000000000000)

In [ ]:
min(df["value"].astype(int)/1000000000000000000)

In [ ]:
from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%m-%d-%Y')
    #.strftime('%m-%d-%Y %H:%M:%S')

df["timeStamp"]

In [ ]:
from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%d-%m-%Y')
    #.strftime('%m-%d-%Y %H:%M:%S')

#df["timeStamp"].apply(convertToDate)


price = []
for i in range(len(df["timeStamp"])):
    data = requests.get(f'https://api.coingecko.com/api/v3/coins/ethereum/history?date={df["timeStamp"].apply(convertToDate)[i]}')
    print(f'https://api.coingecko.com/api/v3/coins/ethereum/history?date={df["timeStamp"].apply(convertToDate)[i]}')
    response = data.json()
    
    #print(response["market_data"])
    price.append(response["market_data"]["current_price"]["usd"])
    
price_df = pd.DataFrame(price)

In [ ]:
price_df = price_df.rename(columns={ 0:"eth price"})
price_df
sum(df["value"] * price_df["eth price"])

In [ ]:
price_df
df

df.join(price_df)


In [ ]:

(sum(df["value"] * price_df["eth price"]))  + sum(((df["gasPrice"].astype(int) * df["gasUsed"].astype(int))/1000000000000000000) * price_df["eth price"])



In [ ]:



#(sum(df["value"] * price_df["eth price"]))  + sum(((df["gasPrice"].astype(int) * df["gasUsed"].astype(int))/1000000000000000000) * price_df["eth price"])


In [ ]:
(sum(df["value"] * price_df["eth price"]))  + sum(((df["gasPrice"].astype(int) * df["gasUsed"].astype(int))/1000000000000000000)

In [ ]:
curl -X 'GET' \
  'https://api.coingecko.com/api/v3/coins/bitcoin/history?date=05-09-2020' \
  -H 'accept: application/json'

In [ ]:

url = "https://api.coingecko.com/api/v3/coins/bitcoin/history?date="

In [ ]:
import datetime
import requests
import sys
import json
requestDate = datetime.datetime(2015,7,30)
endDate = datetime.datetime(2015,12,31)


In [ ]:
headers = { 'Authorization':'Authorization: Bearer '}    
while requestDate <= endDate:
    query = {'date': requestDate.strftime('%Y-%m-%d') }
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        print("{},{:.2f}\n".format(requestDate.strftime('%Y-%m-%d'),ethUSDPrice))
    else:
        print("Response status code {} . Exiting".format(response.status_code))
        print(response.json())
        break
                
    requestDate += datetime.timedelta(days=1)

In [ ]:
from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')
    #.strftime('%m-%d-%Y %H:%M:%S')

#df["timeStamp"].apply(convertToDate)

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
for i in range(len(df["timeStamp"])):
    query = {'date': df["timeStamp"].apply(convertToDate)[i] }
    print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        price.append(ethUSDPrice)
    else:
        print(response.json())
        break

#     data = requests.get(f'https://api.coingecko.com/api/v3/coins/ethereum/history?date={df["timeStamp"].apply(convertToDate)[i]}')
#     print(f'https://api.coinbase.com/v2/prices/ETH-USD/spot', {df["timeStamp"].apply(convertToDate)[i]})
#     response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
#     response = data.json()
    
#     #print(response["market_data"])
#     price.append(response["market_data"]["current_price"]["usd"])
    
# price_df = pd.DataFrame(price)
price_df = pd.DataFrame(price)
price_df

In [ ]:
price_df = price_df.rename(columns={ 0:"eth price"})
price_df


In [ ]:
df_cleaned = pd.merge(price_df, df, left_index=True, right_index=True)
sum(df_cleaned["eth price"] * df_cleaned["value"])


In [ ]:
df_cleaned

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')

def convertToDateFormat(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%m-%d-%Y')

tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
token_url = "https://api.etherscan.io/api?module=account&action=tokennfttx&address=0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
response = requests.get(tx_url)
data = response.json()
txlist_df = pd.DataFrame(data["result"])

response = requests.get(token_url)
data = response.json()
tokennft_df = pd.DataFrame(data["result"])
df = txlist_df.merge(tokennft_df, how='inner', on=["timeStamp"])
dataframe_1 = pd.DataFrame(df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]].groupby(["timeStamp", "tokenSymbol"])["tokenID"].apply(list)).reset_index()
dataframe_2 = df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID", "contractAddress_y"]].drop_duplicates(subset=['timeStamp'], keep='last').reset_index()
df = pd.merge(dataframe_1,dataframe_2,how="inner", on="timeStamp")
df = df.rename(columns={"tokenSymbol_x": "tokenSymbol", "tokenID_x": "tokenID", "tokenSymbol_y": "tokenSymbol", "gas_x": "gas", "gasPrice_x": "gasPrice", "gasUsed_x": "gasUsed" })
df["value"] = df["value"].astype(int)/1000000000000000000

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
date = []
for i in range(len(df["timeStamp"])):
    query = {'date': df["timeStamp"].apply(convertToDate)[i] }
    #print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        date.append(df["timeStamp"].apply(convertToDateFormat)[i])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break


price_df = pd.DataFrame({ "USD": price, "date": date })
df_cleaned = pd.merge(price_df, df, left_index=True, right_index=True)
df_cleaned["gasPriceETH"] = (df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000) 
df_cleaned["gasPriceUSD"] = (df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000) * df_cleaned["USD"]

response = { "data" : df_cleaned.to_dict("records") }
df_cleaned

In [ ]:
from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')
    #.strftime('%m-%d-%Y %H:%M:%S')

#df["timeStamp"].apply(convertToDate)

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
for i in range(len(df["timeStamp"])):
    query = {'date': df["timeStamp"].apply(convertToDate)[i] }
    #print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break

#     data = requests.get(f'https://api.coingecko.com/api/v3/coins/ethereum/history?date={df["timeStamp"].apply(convertToDate)[i]}')
#     print(f'https://api.coinbase.com/v2/prices/ETH-USD/spot', {df["timeStamp"].apply(convertToDate)[i]})
#     response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
#     response = data.json()
    
#     #print(response["market_data"])
#     price.append(response["market_data"]["current_price"]["usd"])
    
# price_df = pd.DataFrame(price)
price_df = pd.DataFrame(price)
price_df

In [ ]:
price_df = price_df.rename(columns={ 0:"dollar price"})
price_df

In [ ]:
df_cleaned = pd.merge(price_df, df, left_index=True, right_index=True)
df_cleaned

In [ ]:


#gas = sum((df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int))/1000000000000000000)

In [ ]:
(df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000) 

In [ ]:

(df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000)  * df_cleaned["dollar price"]

In [ ]:
df_cleaned

In [ ]:
import requests
import pandas as pd

url = "https://api.opensea.io/api/v1/collection/wulfz-official"

response = requests.request("GET", url).json()


pd.DataFrame(response["collection"])

In [ ]:

transaction_url = "https://api.etherscan.io/api?module=account&action=txlistinternal&txhash=0xfac2f3364536035f94df81c804732e5d6f7b3f009c39164ebd8355ca782288a5&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"

response = requests.get(transaction_url)
data = response.json()
data

In [ ]:
import requests
account="0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F"
url = f"https://api.opensea.io/api/v1/events?account_address={account}&limit=300"

headers = {
    "Accept": "application/json",
    "X-API-KEY": "b5e38a68201b473d8a4451dd894d1f03"
}

response = requests.request("GET", url, headers=headers).json()

df = pd.DataFrame(response["asset_events"])
df

In [ ]:
open_df = df[df["event_type"] == "successful"][["asset", "collection_slug", "dev_seller_fee_basis_points", "total_price",  "quantity", "seller", "event_type", "winner_account", "listing_time", "transaction"]]
open_df["royalty_fee"] = (open_df["dev_seller_fee_basis_points"]) / 10000
open_df["opensea_fee"] = 250 / 10000
open_df["salesPriceETH"] = open_df["total_price"].astype(int) / 1000000000000000000

open_df = open_df.reset_index()
transaction_hash_list = []

for i in range(len(open_df["asset"])):
    transaction_hash_list.append(open_df["transaction"][i]["transaction_hash"])
    
#     transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={transaction_hash_list[i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
#     response = requests.get(transaction_url)
#     data = response.json()
#     if(len(data["result"])):
#         print(transaction_hash_list[i])
#         print(data["result"][0]["value"])
#         print(data["result"][1]["value"])
#         print(data["result"])



# # 
# # response = requests.get(transaction_url)
# # data = response.json()
# # data
open_df["transaction_hash"] = transaction_hash_list
open_df

In [ ]:
open_df.reset_index( inplace = True, col_level = 1)


In [ ]:
open_df = unpack(open_df,"asset")


In [ ]:
open_df.columns

In [ ]:
def unpack(df, column, fillna="N/A"):
    ret = None
    if fillna is None:
        tmp = pd.DataFrame((d for idx, d in df[column].iteritems()))
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    else:
        tmp = pd.DataFrame((d for idx, d in 
        df[column].iteritems())).fillna(fillna)
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    return ret

In [ ]:
open_df = unpack(open_df,"winner_account")
open_df = unpack(open_df,"user")
open_df = open_df.rename(columns={"profile_img_url": "winner_profile_img_url", "address": "winner_address", "config": "winner_config", "username": "winner_username"})
open_df = unpack(open_df,"seller")
open_df = unpack(open_df,"user")
open_df = open_df.rename(columns={"profile_img_url": "seller_profile_img_url", "address": "seller_address", "config": "seller_config", "username": "seller_username"})
open_df

In [ ]:
open_df.columns

In [ ]:
df1 = open_df[open_df["seller_address"].isin([account.lower()])]
df2 = open_df[~open_df["seller_address"].isin([account.lower()])]
df1["transaction_type"] = "sale"
df2["transaction_type"] = "buy"
open_df = df1.append(df2)[["collection_slug", "image_preview_url", "token_id", "image_url", "royalty_fee", "opensea_fee", "total_price", "quantity", "salesPriceETH", "winner_profile_img_url", "winner_address", "winner_username", "seller_profile_img_url", "seller_address", "seller_username", "transaction_type", "listing_time", "transaction_hash"]]
opend_df = open_df["listing_time"].replace(to_replace=[None], value="N/A", inplace=True)
#open_df[(open_df["username"] == "N/A") & (open_df["address"] == account.lower())]

In [ ]:
import datetime
price = []
date_list = []
for i in range(len(open_df["listing_time"])):
    date = open_df["listing_time"][i][0:10]
    if(date == "N/A"):
        date = datetime.datetime.now().strftime('%Y-%m-%d')
    query = {'date':date }
    date_list.append(date)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break
    
price_df = pd.DataFrame({ "priceETHtoUSD" : price, "date": date_list })
price_df = price_df.rename(columns={ 0:""})

df_cleaned = pd.merge(price_df, open_df, left_index=True, right_index=True)
df_cleaned["salesPriceUSD"] = df_cleaned["salesPriceETH"] * df_cleaned["priceETHtoUSD"]
df_cleaned["opensea_feeETH"] = df_cleaned["opensea_fee"] * df_cleaned["salesPriceETH"]
df_cleaned["opensea_feeUSD"] = df_cleaned["opensea_feeETH"] * df_cleaned["priceETHtoUSD"]
df_cleaned["royalty_feeETH"] = df_cleaned["royalty_fee"] * df_cleaned["salesPriceETH"]
df_cleaned["royalty_feeUSD"] =  df_cleaned["royalty_feeETH"] * df_cleaned["priceETHtoUSD"]




open_df_sale = df_cleaned[df_cleaned["transaction_type"] == "sale"]
open_df_sale

In [ ]:
open_df_buy = df_cleaned[df_cleaned["transaction_type"] == "buy"]

open_df_buy = open_df_buy.reset_index()
open_df_buy

for i in range(len(open_df_buy['transaction_hash'])):

    transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={open_df_buy['transaction_hash'][i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    print(transaction_url)
    response = requests.get(transaction_url)
    data = response.json()
    print(data["result"])
#     print(open_df_buy['transaction_hash'][i])
#     print(data)
#     transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={transaction_hash_list[i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"

#     if(len(data["result"])):
#         print(transaction_hash_list[i])
#         print(data["result"][0]["value"])
#         print(data["result"][1]["value"])
#         print(data["result"])
#open_df_buy 


In [ ]:
df_cleaned["image_url"][0]

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')

def convertToDateFormat(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%m-%d-%Y')

tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
token_url = "https://api.etherscan.io/api?module=account&action=tokennfttx&address=0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
response = requests.get(tx_url)
data = response.json()
txlist_df = pd.DataFrame(data["result"])

response = requests.get(token_url)
data = response.json()
tokennft_df = pd.DataFrame(data["result"])
df = txlist_df.merge(tokennft_df, how='inner', on=["timeStamp"])
dataframe_1 = pd.DataFrame(df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]].groupby(["timeStamp", "tokenSymbol"])["tokenID"].apply(list)).reset_index()
dataframe_2 = df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID", "contractAddress_y"]].drop_duplicates(subset=['timeStamp'], keep='last').reset_index()
df = pd.merge(dataframe_1,dataframe_2,how="inner", on="timeStamp")
df = df.rename(columns={"tokenSymbol_x": "tokenSymbol", "tokenID_x": "tokenID", "tokenSymbol_y": "tokenSymbol", "gas_x": "gas", "gasPrice_x": "gasPrice", "gasUsed_x": "gasUsed" })
df["value"] = df["value"].astype(int)/1000000000000000000

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
date = []
for i in range(len(df["timeStamp"])):
    query = {'date': df["timeStamp"].apply(convertToDate)[i] }
    #print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        date.append(df["timeStamp"].apply(convertToDateFormat)[i])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break


price_df = pd.DataFrame({ "USD": price, "date": date })
df_cleaned = pd.merge(price_df, df, left_index=True, right_index=True)
df_cleaned["gasPriceETH"] = (df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000) 
df_cleaned["gasPriceUSD"] = (df_cleaned["gasPrice"].astype(int) * df_cleaned["gasUsed"].astype(int)/1000000000000000000) * df_cleaned["USD"]

#response = { "data" : df_cleaned.to_dict("records") }
df_cleaned

In [ ]:
import requests

for i in range(len(open_df)):
    url = f"https://api.opensea.io/api/v1/collection/{open_df['collection_slug'][i]}"
    response = requests.request("GET", url).json()
    print(url)

In [ ]:
from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
for i in range(len(df["timeStamp"])):
    query = {'date': df["timeStamp"].apply(convertToDate)[i] }
    #print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break

#     data = requests.get(f'https://api.coingecko.com/api/v3/coins/ethereum/history?date={df["timeStamp"].apply(convertToDate)[i]}')
#     print(f'https://api.coinbase.com/v2/prices/ETH-USD/spot', {df["timeStamp"].apply(convertToDate)[i]})
#     response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
#     response = data.json()
    
#     #print(response["market_data"])
#     price.append(response["market_data"]["current_price"]["usd"])
    
# price_df = pd.DataFrame(price)
price_df = pd.DataFrame(price)
price_df

In [21]:
import requests
import pandas as pd

account = "0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F"
tx_url = f"https://api.etherscan.io/api?module=account&action=txlist&address={account}&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    
token_url = f"https://api.etherscan.io/api?module=account&action=tokennfttx&address={account}&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"

    

In [22]:
response = requests.get(tx_url)
data = response.json()
txlist_df = pd.DataFrame(data["result"])

response = requests.get(token_url)
data = response.json()
tokennfttx_df = pd.DataFrame(data["result"])

for i in range(len(tokennfttx_df["hash"])):
    transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={tokennfttx_df['hash'][i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    response = requests.get(transaction_url)
    data = response.json()
    print(tokennfttx_df['hash'][i])
    print(data)
    

    
#     response = requests.get(transaction_url)
#     data = response.json()
#     print(data["result"])
#     print(open_df_buy['t

#tokennfttx_df["gas"].astype(int) * tokennfttx_df["gasPrice"].astype(int) /1000000000000000000


0x41b1f76f04aa2ddf2c461572ae52e6a9066a7d9316489a3fa33559742706f3a8
{'status': '0', 'message': 'No transactions found', 'result': []}
0x7964ec5c57aa0990aa2aa1d4758caed5109e826f312b1af5ee50d64153e8d801
{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '13780074', 'timeStamp': '1639174798', 'from': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b', 'to': '0x5b3256965e7c3cf26e11fcaf296dfc8807c01073', 'value': '57900000000000000', 'contractAddress': '', 'input': '', 'type': 'call', 'gas': '2300', 'gasUsed': '0', 'isError': '0', 'errCode': ''}, {'blockNumber': '13780074', 'timeStamp': '1639174798', 'from': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b', 'to': '0xdc8e7537699f344ed3bd861db389c79c92dcc635', 'value': '521100000000000000', 'contractAddress': '', 'input': '', 'type': 'call', 'gas': '2300', 'gasUsed': '0', 'isError': '0', 'errCode': ''}]}
0x1c25acfc3adebd8f194b3ac8744c4dd82ea637f7357bcd429fe2f98c9058e2b9
{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '13831177', 'ti

0x9e8a45fb416bfee1d0a453e4bc844d8b27f157b2b4ee819a443f026164362a19
{'status': '0', 'message': 'No transactions found', 'result': []}
0xa41a5999a6c0467c0fdc13ef4bf7265b3900e8d9def2cf7723b74477b1de2c3c
{'status': '1', 'message': 'OK', 'result': [{'blockNumber': '13908261', 'timeStamp': '1640888658', 'from': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b', 'to': '0x5b3256965e7c3cf26e11fcaf296dfc8807c01073', 'value': '7650000000000000', 'contractAddress': '', 'input': '', 'type': 'call', 'gas': '2300', 'gasUsed': '0', 'isError': '0', 'errCode': ''}, {'blockNumber': '13908261', 'timeStamp': '1640888658', 'from': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b', 'to': '0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f', 'value': '82350000000000000', 'contractAddress': '', 'input': '', 'type': 'call', 'gas': '2300', 'gasUsed': '0', 'isError': '0', 'errCode': ''}]}
0x775f1c3c1a5f65dc199531bf9615d1d827defa30217d302eb523f03309348508
{'status': '0', 'message': 'No transactions found', 'result': []}
0x775f1c3c1

In [ ]:
mint_df = tokennfttx_df[tokennfttx_df["from"] == "0x0000000000000000000000000000000000000000"]
mint_df = txlist_df.merge(mint_df, how='inner', on=["timeStamp"])
mint_df

In [ ]:
buy_df = tokennfttx_df[(tokennfttx_df["to"] == account.lower()) & (tokennfttx_df["from"] != "0x0000000000000000000000000000000000000000")]
buy_df = txlist_df.merge(buy_df, how='inner', on=["timeStamp"])
buy_df




In [ ]:
txlist_df["hash"][7]

#0x7be8076f4ea4a4ad08075c2508e481d6c946d12b
#sale_df = tokennfttx_df[(tokennfttx_df["from"] == account.lower()) & (tokennfttx_df["to"] != account.lower())]
#df1 = pd.DataFrame(sale_df[[ "timeStamp", "tokenSymbol", "tokenName", "tokenID"]].groupby(["timeStamp", "tokenSymbol"])["tokenID"].apply(list)).reset_index()
#txlist_df.merge(df1, how='outer')

#df2 = sale_df[["value", "timeStamp", "tokenSymbol", "gas_x", "gasPrice_x", "gasUsed_x", "tokenName", "tokenID"]].drop_duplicates(subset=['timeStamp'], keep='last').reset_index()

# for i in range(len(txlist_df["hash"])):
#     transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={txlist_df['hash'][i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
#     response = requests.get(transaction_url)
#     data = response.json()
#     if(len(data["result"])):
#         print(txlist_df["hash"][i])

# sale_df = txlist_df.merge(tokennfttx_df, how='outer')
# sale_df

In [ ]:
open_df = txlist_df.merge(tokennfttx_df, how='inner', on="hash")


transaction_hash_list = []

for i in range(len(open_df["hash"])):
    #transaction_hash_list.append(open_df["hash"][i])
    transaction_url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&txhash={open_df['hash'][i]}&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
    response = requests.get(transaction_url)
    data = response.json()
    if(len(data["result"])):
        for d in data['result']:
            print(d["value"])
transaction_hash_list 
open_df

In [ ]:

tokennfttx_df

In [84]:
import requests
import pandas as pd
account="0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F"
url = f"https://api.opensea.io/api/v1/events?account_address={account}&limit=300"

headers = {
    "Accept": "application/json",
    "X-API-KEY": "b5e38a68201b473d8a4451dd894d1f03"
}

response = requests.request("GET", url, headers=headers).json()

event_df = pd.DataFrame(response["asset_events"])
event_df.columns

Index(['approved_account', 'asset', 'asset_bundle', 'auction_type',
       'bid_amount', 'collection_slug', 'contract_address', 'created_date',
       'custom_event_name', 'dev_fee_payment_event',
       'dev_seller_fee_basis_points', 'duration', 'ending_price', 'event_type',
       'from_account', 'id', 'is_private', 'owner_account', 'payment_token',
       'quantity', 'seller', 'starting_price', 'to_account', 'total_price',
       'transaction', 'winner_account', 'listing_time'],
      dtype='object')

In [9]:
def unpack(df, column, fillna="N/A"):
    ret = None
    if fillna is None:
        tmp = pd.DataFrame((d for idx, d in df[column].iteritems()))
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    else:
        tmp = pd.DataFrame((d for idx, d in 
        df[column].iteritems())).fillna(fillna)
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    return ret


In [ ]:
event_df.reset_index( inplace = True, col_level = 1)

In [85]:
transaction_hash_list = []
event_df = event_df[~event_df['transaction'].isna()]
for index, row in event_df.reset_index().iterrows():
    transaction_hash_list.append(row["transaction"]["transaction_hash"])
    
event_df["hash"] = transaction_hash_list
    
# for i, row in event_df[~event_df['transaction'].isna()].reset_index().iterrows():
#     print(row[i])

In [86]:
t_df = tokennfttx_df.merge(event_df, left_on='hash', right_on='hash')
#
t_df.columns

Index(['blockNumber', 'timeStamp', 'hash', 'nonce', 'blockHash', 'from',
       'contractAddress', 'to', 'tokenID', 'tokenName', 'tokenSymbol',
       'tokenDecimal', 'transactionIndex', 'gas', 'gasPrice', 'gasUsed',
       'cumulativeGasUsed', 'input', 'confirmations', 'approved_account',
       'asset', 'asset_bundle', 'auction_type', 'bid_amount',
       'collection_slug', 'contract_address', 'created_date',
       'custom_event_name', 'dev_fee_payment_event',
       'dev_seller_fee_basis_points', 'duration', 'ending_price', 'event_type',
       'from_account', 'id', 'is_private', 'owner_account', 'payment_token',
       'quantity', 'seller', 'starting_price', 'to_account', 'total_price',
       'transaction', 'winner_account', 'listing_time'],
      dtype='object')

In [110]:
test_df = t_df[["tokenName", "tokenID", 'tokenSymbol', "asset", 'total_price', 'timeStamp', 'gas', 'gasPrice', 'gasUsed',
      "event_type",'contractAddress', 'dev_seller_fee_basis_points', "from", "to",  "winner_account"]].fillna(0)

In [156]:
test_df[(test_df["from"] == account.lower()) & (test_df["total_price"] == 0)]

,tokenName,tokenID,tokenSymbol,asset,total_price,timeStamp,gas,gasPrice,gasUsed,event_type,contractAddress,dev_seller_fee_basis_points,from,to,winner_account
6,CoolmansUniverse,6079,COOLMAN,"{'id': 154496679, 'token_id': '6079', 'num_sal...",0,1639879682,303276,107195278655,218748,transfer,0xa5c0bd78d1667c13bfb403e2a3336871396713c5,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x820880c9040041d6a514e069ec0a2f37997325d4,0
10,ALIENFRENS,8045,ALIENFRENS,"{'id': 157035114, 'token_id': '8045', 'num_sal...",0,1640024238,297876,91803581066,214594,transfer,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x47a063d87f988875ab1ded90318bfa497cffc13d,0
15,Wulfz,1579,WULFZ,"{'id': 171559297, 'token_id': '1579', 'num_sal...",0,1640702687,303735,101011201891,219101,transfer,0x9712228ceeda1e2ddde52cd5100b88986d1cb49c,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x8800e42355ebcdde11f7f0d22ef76028bad8e3e9,0
30,Cryptorayrays,1401,rayrays,"{'id': 173415804, 'token_id': '1401', 'num_sal...",0,1640809990,303140,91660017503,218643,transfer,0x8d4e2435c262eb6df10e5e4672a8f07e42d8d67e,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x1b3ca08ba8d6a60bd7250254f873d2d874c8fe89,0
31,Cryptorayrays,1399,rayrays,"{'id': 173415787, 'token_id': '1399', 'num_sal...",0,1640812390,104983,101178721689,95383,transfer,0x8d4e2435c262eb6df10e5e4672a8f07e42d8d67e,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x08ad5be20a99217115c286a672f11c8dae2ade57,0
33,Wulfz,4070,WULFZ,"{'id': 172680905, 'token_id': '4070', 'num_sal...",0,1640855590,353053,84523057904,257038,transfer,0x9712228ceeda1e2ddde52cd5100b88986d1cb49c,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x41db387230fd14cf804474b650eb926331fb97c3,0
39,Wulfz,4788,WULFZ,"{'id': 172738038, 'token_id': '4788', 'num_sal...",0,1640888658,303735,99967155957,219101,transfer,0x9712228ceeda1e2ddde52cd5100b88986d1cb49c,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0x7c590b2243e99103de1cf1acc89d0a3f403bd677,0
45,Kindergarten BabyApes,5885,KBA,"{'id': 177300039, 'token_id': '5885', 'num_sal...",0,1640957044,311288,58201509709,224911,transfer,0x45359616610a584cc39aecdcbb2a457976780023,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0xd139039ac4abe0d6cca88986b4bcc39edb817064,0
47,Kindergarten BabyApes,5886,KBA,"{'id': 177300054, 'token_id': '5886', 'num_sal...",0,1640958746,297937,56687506157,214641,transfer,0x45359616610a584cc39aecdcbb2a457976780023,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0xb919e54a2ecf59fef1d79875ceb858827a5d2d54,0
49,Cryptorayrays,1400,rayrays,"{'id': 173415796, 'token_id': '1400', 'num_sal...",0,1641101496,299406,67652129576,215771,transfer,0x8d4e2435c262eb6df10e5e4672a8f07e42d8d67e,0.0,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,0xee193f18cc3549e1158e4b940f065d8f460c143e,0


In [149]:
mint_df = test_df[test_df["from"] == "0x0000000000000000000000000000000000000000"]
bought_df = test_df[(test_df["to"] == account.lower()) & (test_df["total_price"].astype(int) > 0) ] 
sale_df = test_df[(test_df["to"] != account.lower()) & (test_df["total_price"].astype(int) > 0) ] 
sale_df.reset_index( inplace = True, col_level = 1)
sale_df = unpack(sale_df,"winner_account")
sale_df = unpack(sale_df,"user")
bought_df.reset_index( inplace = True, col_level = 1)
bought_df = unpack(bought_df,"winner_account")
bought_df = unpack(bought_df,"user")
mint_df.reset_index( inplace = True, col_level = 1)
mint_df = unpack(mint_df, "asset")
sale_df = unpack(sale_df, "asset")
bought_df = unpack(bought_df, "asset")
mint_df["type"] = "mint"
bought_df["type"] = "buy"
sale_df["type"] = "sell"



opensea_df = pd.concat([mint_df, sale_df, bought_df], ignore_index=True, sort=False).sort_values(by="timeStamp")


def remove_duplicates(l):
    return list(set(l))


df1 = pd.DataFrame(opensea_df.groupby(["timeStamp", "tokenSymbol", "type"])["tokenID"].apply(remove_duplicates)).reset_index()
opensea_df = pd.merge(df1,opensea_df.drop_duplicates(subset=['timeStamp'], keep='last').reset_index(),how="inner", on="timeStamp")
opensea_df


#.apply(list)).reset_index()
#opensea_df.drop_duplicates(subset=['timeStamp'], keep='last').reset_index()

,timeStamp,tokenSymbol_x,type_x,tokenID_x,level_0,index,tokenName,tokenID_y,tokenSymbol_y,total_price,...,permalink,collection,decimals,token_metadata,owner,type_y,profile_img_url,address,config,username
0,1639174798,COMIC2,buy,[7180],31,1,PUNKS Comic 2,7180,COMIC2,579000000000000000,...,https://opensea.io/assets/0x128675d4fddbc4a0d3...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmeKejc9TNuRKcVqpXRNavF2Z...,"{'user': {'username': None}, 'profile_img_url'...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
1,1639857839,COOLMAN,buy,[6079],32,3,CoolmansUniverse,6079,COOLMAN,170000000000000000,...,https://opensea.io/assets/0xa5c0bd78d1667c13bf...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmbveAod8raJyhDJYJd3E4pAi...,"{'user': {'username': 'eldondys'}, 'profile_im...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
2,1639879682,COOLMAN,sell,[6079],19,5,CoolmansUniverse,6079,COOLMAN,300000000000000000,...,https://opensea.io/assets/0xa5c0bd78d1667c13bf...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmbveAod8raJyhDJYJd3E4pAi...,"{'user': {'username': 'eldondys'}, 'profile_im...",sell,https://storage.googleapis.com/opensea-static/...,0x820880c9040041d6a514e069ec0a2f37997325d4,verified,Wickd
3,1639985237,ALIENFRENS,buy,[8045],33,7,ALIENFRENS,8045,ALIENFRENS,90000000000000000,...,https://opensea.io/assets/0x123b30e25973fecd83...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://gateway.pinata.cloud/ipfs/Qmf5Hzs7xykP...,"{'user': {'username': 'choochootrain'}, 'profi...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
4,1640024238,ALIENFRENS,sell,[8045],20,9,ALIENFRENS,8045,ALIENFRENS,60000000000000000,...,https://opensea.io/assets/0x123b30e25973fecd83...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://gateway.pinata.cloud/ipfs/Qmf5Hzs7xykP...,"{'user': {'username': 'choochootrain'}, 'profi...",sell,https://storage.googleapis.com/opensea-static/...,0x47a063d87f988875ab1ded90318bfa497cffc13d,,Jay4nft
5,1640677409,WULFZ,buy,[1579],34,12,Wulfz,1579,WULFZ,135000000000000000,...,https://opensea.io/assets/0x9712228ceeda1e2ddd...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmYff9fYPHMJkJzbRWXByT3uv...,"{'user': {'username': 'Anon713'}, 'profile_img...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
6,1640702687,WULFZ,sell,[1579],21,14,Wulfz,1579,WULFZ,165000000000000000,...,https://opensea.io/assets/0x9712228ceeda1e2ddd...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmYff9fYPHMJkJzbRWXByT3uv...,"{'user': {'username': 'Anon713'}, 'profile_img...",sell,https://storage.googleapis.com/opensea-static/...,0x8800e42355ebcdde11f7f0d22ef76028bad8e3e9,,eli_V
7,1640722377,WULFZ,buy,[4788],35,16,Wulfz,4788,WULFZ,144000000000000000,...,https://opensea.io/assets/0x9712228ceeda1e2ddd...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmYff9fYPHMJkJzbRWXByT3uv...,"{'user': None, 'profile_img_url': 'https://sto...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
8,1640733945,WULFZ,buy,[4070],36,18,Wulfz,4070,WULFZ,150000000000000000,...,https://opensea.io/assets/0x9712228ceeda1e2ddd...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://ipfs.io/ipfs/QmT6L4BZnfx268aVGem6oygAu...,"{'user': {'username': None}, 'profile_img_url'...",buy,https://storage.googleapis.com/opensea-static/...,0xf717b30d486f3c2ebf77df02714e86d0da6dbd8f,,N/A
9,1640736713,rayrays,mint,"[1399, 1401, 1400]",6,26,Cryptorayrays,1401,rayrays,0,...,https://opensea.io/assets/0x8d4e2435c262eb6df1...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://gateway.pinata.cloud/ipfs/QmRC3nwNmE5e...,"{'user': {'username': 'LylyNFT'}, '

In [81]:
t_df = unpack(t_df,"winner_account")
t_df["winner_account"]
#t_df = unpack(t_df,"user")
# t_df = t_df.rename(columns={"profile_img_url": "winner_profile_img_url", "address": "winner_address", "config": "winner_config", "username": "winner_username"})
# t_df = unpack(t_df,"seller")
# t_df = unpack(t_df,"user")
# t_df = t_df.rename(columns={"profile_img_url": "seller_profile_img_url", "address": "seller_address", "config": "seller_config", "username": "seller_username"})
# t_df

KeyError: 'winner_account'

In [326]:
address="0xF717B30D486F3c2EBF77df02714e86d0da6Dbd8F"
url = f"https://api.opensea.io/api/v1/events?account_address={address}&limit=300"
tx_url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"
token_url = f"https://api.etherscan.io/api?module=account&action=tokennfttx&address={address}&sort=asc&apikey=FKRKV4CZ9A6QGRZRYE5APJV3TP8JPVJIPT"

response = requests.get(tx_url)
data = response.json()
txlist_df = pd.DataFrame(data["result"])
response = requests.get(token_url)
data = response.json()
tokennfttx_df = pd.DataFrame(data["result"])
headers = {
        "Accept": "application/json",
        "X-API-KEY": "b5e38a68201b473d8a4451dd894d1f03"
}

response = requests.request("GET", url, headers=headers).json()

event_df = pd.DataFrame(response["asset_events"])
transaction_hash_list = []
event_df = event_df[~event_df['transaction'].isna()]
for index, row in event_df.reset_index().iterrows():
    transaction_hash_list.append(row["transaction"]["transaction_hash"])
event_df["hash"] = transaction_hash_list
event_df.reset_index( inplace = True, col_level = 1)
t_df = tokennfttx_df.merge(event_df, left_on='hash', right_on='hash')
test_df = t_df[["tokenName", "tokenID", 'tokenSymbol', "asset", 'total_price', 'timeStamp', 'gas', 'gasPrice', 'gasUsed',
      "event_type",'contractAddress', 'dev_seller_fee_basis_points', "from", "to",  "winner_account"]].fillna(0)
mint_df = test_df[test_df["from"] == "0x0000000000000000000000000000000000000000"]
bought_df = test_df[(test_df["to"] == address.lower()) & (test_df["total_price"].astype(int) > 0) ] 
sale_df = test_df[(test_df["to"] != address.lower()) & (test_df["total_price"].astype(int) > 0) ] 

sale_df.reset_index( inplace = True, col_level = 1)
sale_df = unpack(sale_df,"winner_account")


bought_df.reset_index( inplace = True, col_level = 1)
bought_df = unpack(bought_df,"winner_account")


mint_df.reset_index( inplace = True, col_level = 1)
mint_df = unpack(mint_df, "winner_account")

mint_df["type"] = "mint"
bought_df["type"] = "buy"
sale_df["type"] = "sell"


opensea_df = pd.concat([mint_df, sale_df, bought_df], ignore_index=True, sort=False).sort_values(by="timeStamp")

def remove_duplicates(l):
    return list(set(l))


df1 = pd.DataFrame(opensea_df.groupby(["timeStamp", "tokenSymbol", "type"])["tokenID"].apply(remove_duplicates)).reset_index()
opensea_df = pd.merge(df1,opensea_df.drop_duplicates(subset=['timeStamp'], keep='last').reset_index(),how="inner", on="timeStamp")
opensea_df = opensea_df.drop(columns=[0, 'config', "level_0", "index", "event_type"]) 
opensea_df = opensea_df[["timeStamp", "tokenSymbol_x", "type_x", "tokenID_x", "gas", "asset", "total_price", "gasUsed", "gasPrice",  "contractAddress", "user", "from", "to", "profile_img_url", "address", "dev_seller_fee_basis_points"]]


from datetime import datetime
import requests
def convertToDate(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%Y-%m-%d')

def convertToDateFormat(data):
    return datetime.utcfromtimestamp(int(data)).strftime('%m-%d-%Y')

headers = { 'Authorization':'Authorization: Bearer '}  
price = []
dates = []
for i in range(len(opensea_df["timeStamp"])):
    query = {'date': opensea_df["timeStamp"].apply(convertToDate)[i] }
    dates.append(opensea_df["timeStamp"].apply(convertToDateFormat)[i])
    #print(query)
    response = requests.get('https://api.coinbase.com/v2/prices/ETH-USD/spot', params=query,headers=headers)
    if response.status_code==200:
        data = response.json()
        ethUSDPrice = float(data['data']['amount'])
        price.append(ethUSDPrice)
    else:
        #print(response.json())
        break



price_df = pd.DataFrame({ "ETHtoUSD": price, "date": dates})
df_cleaned = pd.merge(price_df, opensea_df, left_index=True, right_index=True)
df_cleaned["gasPriceETH"] = (opensea_df["gasPrice"].astype(int) * opensea_df["gasUsed"].astype(int)/1000000000000000000) 
df_cleaned["salePriceETH"] = (opensea_df["total_price"].astype(int)/1000000000000000000)
#df_cleaned["salePriceUSD"] = (opensea_df["salePriceETH"].astype(int) * opensea_df["ETHtoUSD"].astype(float))
df_cleaned["gasPriceUSD"] = (df_cleaned["gasPriceETH"].astype(int) * df_cleaned["ETHtoUSD"])
df_cleaned["salePriceUSD"] = (df_cleaned["salePriceETH"].astype(float) * df_cleaned["ETHtoUSD"].astype(float))



In [329]:

df_cleaned["asset"][0]



{'id': 140226067,
 'token_id': '7180',
 'num_sales': 1,
 'background_color': None,
 'image_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8',
 'image_preview_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8=s250',
 'image_thumbnail_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8=s128',
 'image_original_url': 'https://ipfs.io/ipfs/QmUSDxiPRFSf22yCda4Wecd1xpZcaZrugKTWBBUYXPTXDN/',
 'animation_url': None,
 'animation_original_url': None,
 'name': 'PUNKS 2: X Marks the Drop',
 'description': 'PUNKS Comic Issue #2: Our tale resumes as Courtney and the PUNKS must work alongside the 8 Elite Apes to defeat the nefarious Frank Lopez and score a legendary bounty. This tale features a new Ape on the scene, Indigo

In [325]:
df_cleaned["profile_img_url"] = df_cleaned["profile_img_url"].fillna("None")
df_cleaned["address"] = df_cleaned["address"].fillna("None")
image_url_list = []
name_list = []
for i, row in df_cleaned.iterrows():
    if(row["asset"]):
        image_url_list.append(row["asset"]["image_url"])
        name_list.append(row["asset"]["name"])
    else: 
        image_url_list.append("None")
        name_list.append("None")
df_cleaned["nft_image_url"] = name_list
df_cleaned["nft_collection_name"] = image_url_list
df_cleaned["royaltyFee"] = df_cleaned["dev_seller_fee_basis_points"]/10000
df_cleaned["royaltyFeeUSD"] = df_cleaned["salePriceUSD"] *  df_cleaned["royaltyFee"]
df_cleaned["royaltyFeeETH"] = df_cleaned["salePriceETH"] *  df_cleaned["royaltyFee"]
df_cleaned[["date","tokenSymbol_x", "timeStamp",  "type_x", "tokenID_x", "gasUsed",
            "gasPrice", "contractAddress", "from", "to", "address", "gasPriceETH", "salePriceETH", "nft_collection_name"
            "gasPriceUSD", "salePriceUSD", "nft_image_url", "royaltyFee", "royaltyFeeUSD", "royaltyFeeETH"]]

df_cleaned.to_dict("record")

#sale_df[~sale_df["tokenName"].isnull()]

/Users/drewhoang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'ETHtoUSD': 4105.99,
  'date': '12-10-2021',
  'timeStamp': '1639174798',
  'tokenSymbol_x': 'COMIC2',
  'type_x': 'buy',
  'tokenID_x': ['7180'],
  'gas': '301667',
  'asset': {'id': 140226067,
   'token_id': '7180',
   'num_sales': 1,
   'background_color': None,
   'image_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8',
   'image_preview_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8=s250',
   'image_thumbnail_url': 'https://lh3.googleusercontent.com/6SiPy_-gzM9pmk2_bQO2UQUYWc7ccnT8QljFNjopVggbTP2qD42stDk8ViCWcoP7c013J8PYSkFK8Pmpc4Bp_OT9eRsnj7X63IcalN8=s128',
   'image_original_url': 'https://ipfs.io/ipfs/QmUSDxiPRFSf22yCda4Wecd1xpZcaZrugKTWBBUYXPTXDN/',
   'animation_url': None,
   'animation_original_url': None,
   'name': 'PUNKS 2: X Marks the Drop',
   'description': 'PUNKS Comic Issu

In [258]:
requests.get("").json()

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/nft/0xb1332307Bd818Db68e372269FD515F6523cd4BCE (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7e2441cd0>: Failed to establish a new connection: [Errno 61] Connection refused'))